<a href="https://colab.research.google.com/github/chelsk5/UW-LING-573-Group-5/blob/main/LING573_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Authors: Nina Koh, Chelsea Kendrick
# Setup: Clone repo & install dependencies

!rm -rf UW-LING-573-Group-5
!git clone https://github.com/chelsk5/UW-LING-573-Group-5.git

!pip install -r UW-LING-573-Group-5/requirements.txt

Cloning into 'UW-LING-573-Group-5'...
remote: Enumerating objects: 482, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 482 (delta 15), reused 16 (delta 10), pack-reused 457 (from 1)
Receiving objects: 100% (482/482), 352.84 KiB | 10.08 MiB/s, done.
Resolving deltas: 100% (81/81), done.


In [2]:
%cd UW-LING-573-Group-5/
%ls

/content/UW-LING-573-Group-5
LING573_Model.ipynb  README.md         rouge_eval.py   tf-idf.py
Minutes.md           requirements.txt  summaries.json


In [3]:
!pip uninstall -y torchvision # bug

In [4]:
# 1. Load data
from datasets import load_dataset
opinosis = load_dataset("kavgan/opinosis", split="train") # HuggingFace hub
dataset = opinosis.train_test_split(test_size=0.2,seed=1) # follow common split ratio
train_data = dataset["train"]
test_data = dataset["test"]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/datasets/load.py:1486: FutureWarning: The repository for kavgan/opinosis contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/kavgan/opinosis
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.

In [5]:
# 2. Preprocessing
def clean_data(sent_to_sum):
  """Cleans the given review and returns a dictionary
  of aggregated reviews and the first gold summary"""
  reviews = sent_to_sum["review_sents"].strip()
  # Choose the first gold summary
  gold_summary = sent_to_sum["summaries"][0].strip()
  return {
      "reviews": reviews,
      "gold_summary": gold_summary
  }

# Sanity check!
# Apply cleaning to a small sample
cleaned_train = train_data.map(clean_data)
cleaned_test = test_data.map(clean_data)

# Print the result
print("Input reviews:")
print(cleaned_train[0]["reviews"])
print("\nTarget summary:")
print(cleaned_train[0]["gold_summary"])

print("\nInput reviews:")
print(cleaned_train[1]["reviews"])
print("\nTarget summary:")
print(cleaned_train[1]["gold_summary"])

Input reviews:
The food for our event was delicious .
 The food in the lounge was great and very fresh, , , salads, sandwiches etc .
 As far as food, walk a few blocks toward Michigan Ave turn left or right and there are plently of less expensive places to eat .
 The Palm resturant in the hotel had some specials Sunday night, we ate there and the food service,etc were outstanding portions are large and we shared since we are not big eaters .
 Took the charge of the minibar which we had used to keep my 2 year old sons food .
 We never ate anything onsite so I can't vouch for the food options immediately avialable .
 The Lobby bar does not serve food very late at night and we couldn't find any vending machines for soda or snacks, so stop at a nearby market before going in for the night .
 In any case, I had allotted approximately $150 for food for 4 days 3 nights .
 But the presentation was awesome and the food was so good and not having to walk in the dark at night and scare my mother

In [6]:
# 3. Load tokenizer & model
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _

In [7]:
# 4. Tokenize reviews & gold summaries
def tokenize_data(cleaned_data):
  """Convert the raw text into token IDs for model processing"""
  # Prepend task for T5 to execute
  pref_text = "summarize: " + cleaned_data["reviews"]

  # Tokenize input (reviews)
  # Ensure that text does not exceed (is in fact exactly) maximum of 512 tokens
  tokenized_input = tokenizer(pref_text, max_length=512, truncation=True, padding="max_length")

  # Tokenize target output (gold summary)
  gold_summary = cleaned_data["gold_summary"]
  labels = tokenizer([gold_summary], max_length=64, truncation=True) # expects a list

  # Add target labels to the model input
  # Format for HuggingFace Trainer
  tokenized_input["labels"] = labels["input_ids"][0] # get first (& only) item of nested list
  return tokenized_input

#apply tokenization to data
tokenized_train = cleaned_train.map(tokenize_data, batched=False) # one row at a time
tokenized_test = cleaned_test.map(tokenize_data, batched=False)

#check tokenization
print("Tokenized input example (input_ids):")
print(tokenized_train[0]["input_ids"])
print("Tokenized target (labels):")
print(tokenized_train[0]["labels"])



Tokenized input example (input_ids):
[21603, 10, 37, 542, 21, 69, 605, 47, 3326, 3, 5, 37, 542, 16, 8, 9658, 47, 248, 11, 182, 1434, 6, 3, 6, 3, 6, 5870, 7, 6, 20017, 672, 3, 5, 282, 623, 38, 542, 6, 1482, 3, 9, 360, 6438, 2957, 5847, 8945, 919, 646, 42, 269, 11, 132, 33, 4752, 295, 120, 13, 705, 2881, 1747, 12, 3, 1544, 3, 5, 37, 8005, 880, 450, 288, 16, 8, 1595, 141, 128, 534, 7, 1771, 706, 6, 62, 3, 342, 132, 11, 8, 542, 313, 6, 21117, 130, 4590, 17622, 33, 508, 11, 62, 2471, 437, 62, 33, 59, 600, 23852, 7, 3, 5, 304, 1825, 8, 1567, 13, 8, 3016, 1047, 84, 62, 141, 261, 12, 453, 82, 204, 215, 625, 520, 7, 542, 3, 5, 101, 470, 3, 342, 959, 3, 24452, 78, 27, 54, 31, 17, 3, 1621, 2295, 21, 8, 542, 931, 2017, 3, 2960, 138, 179, 3, 5, 37, 17464, 969, 1207, 405, 59, 1716, 542, 182, 1480, 44, 706, 11, 62, 2654, 31, 17, 253, 136, 19518, 53, 4096, 21, 12495, 42, 12751, 6, 78, 1190, 44, 3, 9, 4676, 512, 274, 352, 16, 21, 8, 706, 3, 5, 86, 136, 495, 6, 27, 141, 66, 14708, 26, 3241, 25689, 21, 3

In [8]:
# 5. Create batch of examples
from transformers import DataCollatorForSeq2Seq

# Dynamically pad sentences to the longest length
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [10]:
# 6. Evaluate model using ROUGE
import evaluate, numpy as np

rouge = evaluate.load("rouge")

def compute_rouge(eval_pred):
  """Decodes token IDs and computes the ROUGE score of generated summaries"""
  predictions, labels = eval_pred # model outputs & gold summaries as token IDs
  # Decode model output IDs back into text
  decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

  # Replace padding tokens IDs (-100) with tokenizer's pad token ID (usually 0)
  labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
  # Decode gold summary IDs back into text
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  # Compare model output against gold labels via ROUGE
  result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
  clean_result = {k: round(v, 4) for k,v in result.items()} # map rouge type to value
  return clean_result

FileNotFoundError: Couldn't find a module script at /content/UW-LING-573-Group-5/rouge/rouge.py. Module 'rouge' doesn't exist on the Hugging Face Hub either.

In [ ]:
# 7. Train model
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./t5-small-results", # folder to save progress
    evaluation_strategy="epoch", # evaluate after one complete pass
    learning_rate=2e-5,
    per_device_train_batch_size=8, # small batch size for small data
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3, # number of checkpoints (subject to change?)
    num_train_epochs=3, # total passes thru data (also stc...)
    predict_with_generate=True,
    fp16=True,
)

# Initialize the trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_rouge
)

# Train (finally)
trainer.train()